# 2-3 动态计算图
本节我们将介绍Pytorch的动态计算图。
包括：
* 动态计算图简介
* 计算图中的Function
* 计算图和反向传播
* 叶子节点和非叶子节点
* 计算图在TensorBoard中的可视化

## 一、动态计算图简介
![](./data/torch动态图.gif)

Pytorch的计算图由节点和边组成，节点表示张量或者Function，边表示张量和Function之间的依赖关系。

Pytorch中的计算图是动态图。这里的动态主要有两重含义。

* 第一重含义是：计算图的正向传播是立即执行的。无需等待完整的计算图创建完毕，每条语句都会在计算图中动态添加节点和边，并立即执行正向传播得到计算结果。
* 第二重含义是：计算图在反向传播后立即销毁。下次调用需要重新构建计算图。如果程序中使用了backward方法执行了反向传播，或者利用torch.autograd.grad方法计算了梯度，那么创建的计算图会被立即销毁，释放储存空间，下次调用需要重新创建。

### 1.计算图的正向传播是立即执行的

In [1]:
import torch
w = torch.tensor([[3.0, 1.0]], requires_grad=True)
b = torch.tensor([[3.0]], requires_grad=True)
x = torch.randn(10, 2)
y = torch.randn(10, 1)
y_hat = x@w.t() + b

loss = torch.mean(torch.pow(y_hat-y, 2))
print(loss.detach())
print(y_hat.detach())

tensor(12.0878)
tensor([[ 1.3707],
        [ 3.6980],
        [ 5.1801],
        [ 1.0187],
        [ 3.3648],
        [ 3.5799],
        [ 2.2841],
        [-1.1161],
        [ 4.2858],
        [ 1.5342]])


### 2.计算图在反向传播后立即销毁

In [3]:
# backward调用之后, 立即销毁计算图
loss.backward()  # 如果需要保留的话, 需要设置retain_graph = True

# loss.backward()  # 再次执行就会报错

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

## 二、计算图中的Function
计算图中一般有着两种重要的节点, 一种是Tensor另一种就是Function, 实际上就是Pytorch中各种对张量操作的函数。

这些Function和Python函数有一个重要的区别就是, 这里的Function同时包含了正向计算和反向传播的逻辑。

我们可以通过继承torch.autograd.Function来创建这种支持反向传播的Function

In [4]:
class MyReLU(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        # 这里的grad_out就是upstream gradient
        # 所以为了得到downstream gradient就只需要在这个函数里面计算local gradient
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [5]:
w = torch.tensor([[3., 1.]], requires_grad=True)
b = torch.tensor([[3.0]], requires_grad=True)
x = torch.tensor([[-1., -1.], [1., 1.]])
y = torch.tensor([[2., 3]])

myrelu = MyReLU.apply  # 函数句柄
y_hat = myrelu(x@w.t() + b)
loss = torch.mean(torch.pow(y_hat-y, 2))

loss.backward()

print(w.grad)
print(b.grad)

tensor([[4.5000, 4.5000]])
tensor([[4.5000]])


In [6]:
print(y_hat.grad_fn)

## 三、计算图与反向传播
了解了Function的功能, 我们可以简单地理解一下反向传播的原理和过程。理解该部分原理需要一些高等数学的基础知识。

In [10]:
from functools import partial
x = torch.tensor(3.0, requires_grad=True)
y1 = x + 1
y2 = 2*x
loss = (y1-y2)**2
# 注册钩子函数看的更清楚
def f(name, grad):
    print(name+"_grad:", grad)
y1_f = partial(f, "y1")
y2_f = partial(f, "y2")
y1.register_hook(y1_f)
y2.register_hook(y2_f)

loss.backward()
print("x_grad:", x.grad)

y2_grad: tensor(4.)
y1_grad: tensor(-4.)
x_grad: tensor(4.)


loss.backward()语句调用之后, 以此发生以下计算过程：

1. loss(是一个scalar)把自己的grad梯度赋值为1, 也就是loss在计算图中的upstream gradient

2. loss根据自身梯度以及关联的backward方法, 计算出其对应的自变量即y1和y2的梯度, 将该梯度赋值到y1.grad和y2.grad(后续会被删除)

3. y2和y1根据自身梯度以及关联的backward方法, 分别计算出对应的自变量x的梯度, x.grad将其收到的多个梯度值进行累加。


**正是因为求导链式法则衍生的梯度累加规则, 张量的grad梯度不会自动清零, 在需要的时候需要手动设置零**

In [14]:
# 下面演示梯度清0的操作
from functools import partial
x = torch.tensor(3.0, requires_grad=True)
y1 = x + 1
y2 = 2*x
loss = (y1-y2)**2

for i in range(2):
    loss.backward(retain_graph=True)
    # 1.如果不清0, 会输出4, 8
    # 2.直接清0: x.grad.zero_()
    # 3.使用detach
    grad = x.grad.detach_()
    grad.zero_()
    print(x.grad)

tensor(0.)
tensor(0.)


## 四、叶子节点和非叶子节点
执行下面代码, 我们会发现loss.grad并不是我们期望的1, 而是None。

类似地y1.grad和y2.grad也是None。

这是由于它们不是叶子节点张量。

在反向传播过程中, 只有is_leaf=True的叶子节点, 且需要求导的张量的导数结果才会被最后保留下里。

叶子节点张量一般需要满足两个条件：

1. 一般requires_grad为False就默认为叶子节点张量(requires_grad为False意味着在这次backward中并没有什么需要计算的依赖)。

2. 由用户创建的requires_grad为True的张量也是叶子节点丈量。

Pytorch这样设计的主要目的是为了节省内存或者显存空间。

如果需要保留中间计算结果的梯度到grad属性中，可以使用 retain_grad方法。 如果仅仅是为了调试代码查看梯度值，可以利用register_hook打印日志

In [15]:
x = torch.tensor(3.0, requires_grad=True)  # 用户创建的requires_grad为True的叶子节点

y1 = x + 1  # 不是叶子节点, 但是由于依赖x, 所以requires_grad自动被设置为True

y2 = 2*x  # 和y1一样

loss = (y1-y2)**2

loss.backward()
print("loss.grad:", loss.grad)
print("y1.grad:", y1.grad)
print("y2.grad", y2.grad)
print(x.grad)

loss.grad: None
y1.grad: None
y2.grad None
tensor(4.)


In [16]:
print("x:", x.is_leaf, x.requires_grad)
print("y1:", y1.is_leaf, y1.requires_grad)
print("y2:", y2.is_leaf, y2.requires_grad)
print("loss:", loss.is_leaf, loss.requires_grad)

x: True True
y1: False True
y2: False True
loss: False True


In [17]:
# 使用retain_grad保留非叶子节点的梯度
x = torch.tensor(3.0, requires_grad=True)
y1 = x + 1
y2 = 2*x
loss = (y1-y2)**2
# 保留loss的grad
loss.retain_grad()

loss.backward()
print("loss.grad:", loss.grad)
print(x.grad)

loss.grad: tensor(1.)
tensor(4.)


## 五、计算图在TensorBoard中可视化

In [20]:
from torch import nn
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.randn(2, 1))
        self.b = nn.Parameter(torch.zeros(1, 1))

    def forward(self, x):
        y = x@self.w + self.b
        return y

In [26]:
from torch.utils.tensorboard import SummaryWriter
net = Net()
logdir = "./data/tensorboard/2_3_autograph"
writer = SummaryWriter(logdir)
writer.add_graph(net, input_to_model=torch.rand(10, 2))
writer.close()

In [24]:
from tensorboard import notebook
notebook.list() 
#在tensorboard中查看模型
notebook.start("--logdir " + logdir)

No known TensorBoard instances running.
